In [1]:
#libraries
import re
import pandas as pd
import numpy as np
from scipy.stats import chisquare
import pysal.model.spreg as spreg
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pysal.lib
import geopandas as gpd
import pandas as pd
from libpysal.weights import DistanceBand
from scipy.spatial import distance_matrix
import pandas as pd
from scipy.stats import pearsonr
from esda.moran import Moran
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import spreg
from spreg import SURlagIV
from spreg import GM_Lag
import libpysal
from spreg import AKtest
from spreg import ML_Lag
from spreg import GM_Combo

**Loading Path**

In [2]:
# Dependent variable
dep_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\2.2 Saving the Dependent Variables"

# Independent variable
ind_var_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\1.2 Saving the Independent Variables"

# Steamengines and horsepower
power_path = r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data"

In [3]:
# lets read the metadata
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\control.dta") as reader:
    meta_control = reader.variable_labels()
    
with pd.io.stata.StataReader(r"C:\Users\soere\OneDrive\Desktop\Python MA\0. Pretransformed Data\_FINAL_c_steamenginges_1904.dta") as reader:
    meta_steam = reader.variable_labels()

In [4]:
meta_control

{'id_inequality': 'County ID',
 'rb_id': 'District ID',
 'rb_name': 'District name',
 'province_id': 'Province ID',
 's_industry': 'Share industry employment',
 's_prot': 'Share Protestants',
 's_german': 'Share German-speaking',
 'longitud': 'Longitude',
 'latitud': 'Latitude',
 'slope_km': 'Terrain slope',
 'soil_suitability': 'Soil suitability',
 'mean_temp': 'Average temperature',
 'mean_preci': 'Average precipitation',
 'dist_100km': 'Distance to the nearest river',
 'log_distance_carbon_c': 'Least distance to carbon field (in log)'}

In [5]:
# Load Skill levels per Landkreis
skills_df = pd.read_csv(f"{dep_var_path}\Main_Dep.csv",
                        sep=",")

# Load control variables
control_df = pd.read_csv(f"{ind_var_path}\Independent Variables.csv",
                         sep=",")

# Load SteamEngine and Horsepower data
power_df = pd.read_stata(f"{power_path}\_FINAL_c_steamenginges_1904.dta")

In [6]:
# Merge the dataframes using an inner join on the "id_inequality" column
prep_df = pd.merge(control_df, skills_df, on="id_inequality", how="inner")
merged_df = pd.merge(prep_df, power_df, on="id_inequality", how="inner")

In [7]:
# Transform steamengines to steamengines per worker
merged_df["steamengines_number"] = (merged_df["steamengines_number"] / merged_df["d_Hauptberuf"])*1000
merged_df["horsepower"] = (merged_df["horsepower"] / merged_df["d_Hauptberuf"])*1000

merged_df["steamengines_number"] = merged_df["steamengines_number"]
merged_df["horsepower"] = np.log(merged_df["horsepower"])

In [8]:
merged_df["per_unskilled"] = merged_df["per_unskilled"]*100
merged_df["per_low"] = merged_df["per_low"]*100
merged_df["per_medium"] = merged_df["per_medium"]*100
merged_df["per_high"] = merged_df["per_high"]*100

In [9]:
# Drop Erblanden
#district_Erblanden = [31]
#merged_df = merged_df[~merged_df["rb_id"].isin(district_Erblanden)]

# Make Dummies for FE

In [10]:
dummies = pd.get_dummies(merged_df["province_id"], drop_first=True) * 1
merged_df = pd.concat([merged_df, dummies], axis=1)

# Convert column names to strings
merged_df.columns = [str(col) for col in merged_df.columns]

In [11]:
# Dictionary
number_words = {
    "1": "one", "2": "two", "3": "three", "4": "four", "5": "five",
    "6": "six", "7": "seven", "8": "eight", "9": "nine", "10": "ten",
    "11": "eleven", "12": "twelve", "13": "thirteen", "14": "fourteen"
}

# Transform the column names
merged_df.columns = [number_words[col.split('.')[0]] if col.split('.')[0] in number_words else col for col in merged_df.columns]

merged_df

,id_inequality,rb_id,rb_name_x,province_id,s_industry,s_prot,s_german,longitud,latitud,slope_km,...,five,six,seven,eight,nine,ten,eleven,twelve,thirteen,fourteen
0,1.0,17.0,Koenigsberg,1.0,18.515817,93.379268,56.527919,21.261999,55.648117,0.039,...,0,0,0,0,0,0,0,0,0,0
1,2.0,17.0,Koenigsberg,1.0,20.088563,96.752059,99.449711,20.249264,54.843463,0.041,...,0,0,0,0,0,0,0,0,0,0
2,4.0,17.0,Koenigsberg,1.0,35.498406,91.659959,99.395196,20.503550,54.706534,0.041,...,0,0,0,0,0,0,0,0,0,0
3,5.0,17.0,Koenigsberg,1.0,15.060419,97.782661,99.669951,20.646094,54.737281,0.039,...,0,0,0,0,0,0,0,0,0,0
4,6.0,17.0,Koenigsberg,1.0,12.007056,98.668155,89.094425,21.267903,54.855057,0.038,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,698.0,1.0,Aachen,12.0,15.278715,1.613553,70.916132,6.167253,50.364714,0.143,...,0,0,0,0,0,0,0,1,0,0
571,699.0,31.0,Sigmaringen,14.0,19.492555,5.190840,99.167226,9.235305,48.006617,0.165,...,0,0,0,0,0,0,0,0,0,1
572,700.0,31.0,Sigmaringen,14.0,23.974037,2.518404,99.882758,9.198332,48.235600,0.186,...,0,0,0,0,0,0,0,0,0,1
573,701.0,31.0,Sigmaringen,14.0,30.325834,6.910838,99.726559,9.001089,48.317343,0.182,...,0,0,0,0,0,0,0,0,0,1


# 2SLS
Lets run a good'ol reliable IV first

## 2SLS function
I build a function that runs multiple 2SLS in a row, each time adding another variable from a list of control variables

In [12]:
def twoSLS(df, IV, first_stage_dependent, controls, second_stage_independent):
    """
    Performs a series of Two-Stage Least Squares (2SLS) regressions on a given DataFrame.
    In each iteration, an additional control variable from the provided list is included.

    Parameters:
    df (DataFrame): The DataFrame containing the data for regression.
    IV (str): The name of the instrumental variable column in the DataFrame.
    first_stage_dependent (str): The name of the dependent variable for the first stage regression.
    controls (list of str): A list of control variable names. Each iteration includes an additional control variable.
    second_stage_independent (str): The name of the main independent variable for the second stage regression.

    Returns:
    tuple of DataFrames: The first DataFrame contains the results (coefficients, significance, 
                         Adj. R-squared, F-statistic) of each first stage regression. 
                         The second DataFrame contains the results 
                         of each second stage regression. Significance is denoted by stars: 
                         *** for p < 0.01, ** for p < 0.05, and * for p < 0.1.
                         The third DataFrame contains the residuals.
    """

    first_stage_results = []
    second_stage_results = []
    # For analysis of spatial autocorrelation
    residuals = []

    for i in range(len(controls) + 1):
        # Subset of control variables for this iteration
        subset_controls = controls[:i]

        # Build the formula, handling the case with no controls
        controls_formula_part = " + ".join(subset_controls)
        if controls_formula_part:
            controls_formula_part = " + " + controls_formula_part

        # To avoid SettingWithCopyWarning
        df_copy = df.copy()

        # First Stage Regression
        first_stage_formula = f"{first_stage_dependent} ~ {IV}{controls_formula_part}"
        first_stage_model = smf.ols(first_stage_formula, data=df_copy)
        first_stage_fit = first_stage_model.fit()

        # Add the predicted values as a new column in the copy
        df_copy["predicted"] = first_stage_fit.predict(df_copy)

        # Second Stage Regression
        second_stage_formula = f"{second_stage_independent} ~ predicted{controls_formula_part}"
        second_stage_model = smf.ols(second_stage_formula, data=df_copy)
        second_stage_fit = second_stage_model.fit()

        def format_coef(coef, std_err, p_value):
            stars = ""
            if p_value < 0.01:
                stars = "***"
            elif p_value < 0.05:
                stars = "**"
            elif p_value < 0.1:
                stars = "*"
    
            # Check if coef or std_err is smaller than 0.000 and format accordingly
            coef_str = f"{coef:.10e}" if abs(coef) < 0.000 else f"{coef:.3f}"
            std_err_str = f"{std_err:.10e}" if abs(std_err) < 0.000 else f"{std_err:.3f}"

            return f"{coef_str}{stars} ({std_err_str})"


        # Extract coefficients and significance of first stage
        first_stage_summary = first_stage_fit.summary2().tables[1]
        first_stage_coefs = first_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Extract coefficients and significance of second stage
        second_stage_summary = second_stage_fit.summary2().tables[1]
        second_stage_coefs = second_stage_summary.apply(lambda row: format_coef(row["Coef."], row["Std.Err."], row["P>|t|"]), axis=1)

        # Adding R-squared and F-statistic
        first_stage_coefs.loc["Adj. R-squared"] = f"{first_stage_fit.rsquared_adj:.3f}"
        first_stage_coefs.loc["F-statistic"] = f"{first_stage_fit.fvalue:.3f}"
        second_stage_coefs.loc["Adj. R-squared"] = f"{second_stage_fit.rsquared_adj:.3f}"
        
        # Adding Standard Errors
        second_stage_se = second_stage_fit.bse["predicted"]
        second_stage_coefs.loc["std err"] = second_stage_se
        
        # Residuals
        res = second_stage_fit.resid
        
        # Append 
        first_stage_results.append(first_stage_coefs)
        second_stage_results.append(second_stage_coefs)
        residuals.append(res)

    # Convert the results to DataFrame
    first_stage_df = pd.DataFrame(first_stage_results).T
    second_stage_df = pd.DataFrame(second_stage_results).T
    
    # Df of residuals
    residuals_df = pd.DataFrame(residuals).T

    return first_stage_df, second_stage_df, residuals_df

## Use the Function

In [13]:
# DataFrame that contains all the variables for the regression analysis
dep_var = merged_df[["log_distance_carbon_c", 
                     "s_industry",
                     "s_prot", 
                     "s_german", 
                     "longitud", 
                     "latitud", 
                     "soil_suitability", 
                     "mean_temp", 
                     "mean_preci", 
                     "dist_100km",
                     "uni_dummy", 
                     "min_distance_uni", 
                     "prov_cap_dummy", 
                     "BER_dummy", 
                     "density",
                     "steamengines_number",
                     "per_low",
                     "per_medium",
                     "per_unskilled",
                     "per_high",
                     "d_Hauptberuf",
                     "horsepower",
                     "two", 
                     "three", 
                     "four", 
                     "five", 
                     "six", 
                     "seven", 
                     "eight", 
                     "nine", 
                     "ten", 
                     "eleven", 
                     "twelve", 
                     "thirteen", 
                     "fourteen"]]

In [14]:
# Make two lists of exogenous variables
# Exclude latitude and longitude once
control_list = ["longitud",
                "latitud",
                "soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "two", 
                "three", 
                "four", 
                "five", 
                "six", 
                "seven", 
                "eight", 
                "nine", 
                "ten", 
                "eleven", 
                "twelve", 
                "thirteen", 
                "fourteen"]

control_list_2 = ["soil_suitability", 
                "mean_temp", 
                "mean_preci", 
                "dist_100km",
                "s_prot", 
                "s_german",
                "density",
                "min_distance_uni", 
                "prov_cap_dummy", 
                "two", 
                "three", 
                "four", 
                "five", 
                "six", 
                "seven", 
                "eight", 
                "nine", 
                "ten", 
                "eleven", 
                "twelve", 
                "thirteen", 
                "fourteen"]

In [15]:
first_unskilled, second_unskilled, res_unskilled = twoSLS(dep_var,
                                                          "log_distance_carbon_c",
                                                          "steamengines_number",
                                                          control_list,
                                                          "per_unskilled")

first_low, second_low, res_low = twoSLS(dep_var,
                                        "log_distance_carbon_c",
                                        "steamengines_number",
                                        control_list,
                                        "per_low")

first_medium, second_medium, res_medium = twoSLS(dep_var,
                                                 "log_distance_carbon_c",
                                                 "steamengines_number",
                                                 control_list,
                                                 "per_medium")

first_high, second_high, res_high = twoSLS(dep_var,
                                           "log_distance_carbon_c",
                                           "steamengines_number",
                                           control_list,
                                           "per_high")

In [16]:
first_unskilled[[1,2,3,24]]

,1,2,3,24
Intercept,18.720*** (1.274),-78.074*** (19.669),-82.289*** (19.505),-50.707 (44.626)
log_distance_carbon_c,-1.466*** (0.280),-2.595*** (0.357),-2.638*** (0.354),-2.160*** (0.423)
Adj. R-squared,0.053,0.090,0.109,0.218
F-statistic,17.180,20.025,18.520,7.665
longitud,-0.028 (0.093),-0.122 (0.093),-0.181* (0.094),0.794* (0.441)
latitud,NaN,1.961*** (0.398),2.123*** (0.396),0.364 (0.821)
soil_suitability,NaN,NaN,-1.393*** (0.390),-1.492*** (0.401)
mean_temp,NaN,NaN,NaN,4.620*** (0.806)
mean_preci,NaN,NaN,NaN,0.162*** (0.060)
dist_100km,NaN,NaN,NaN,-1.288 (2.119)


In [17]:
# First Stage (same for every regression)
second_unskilled[[1,2,3,24]]

,1,2,3,24
Intercept,59.619*** (3.872),-7.187 (19.152),-10.153 (19.153),-64.343 (49.844)
predicted,-1.307*** (0.231),-0.845*** (0.170),-0.828*** (0.167),-0.786*** (0.226)
Adj. R-squared,0.153,0.153,0.152,0.363
std err,0.230728,0.169767,0.167222,0.225685
longitud,0.590*** (0.115),0.546*** (0.120),0.489*** (0.126),-0.235 (0.536)
latitud,NaN,1.178*** (0.377),1.309*** (0.381),3.341*** (0.921)
soil_suitability,NaN,NaN,-1.410*** (0.532),-1.213** (0.575)
mean_temp,NaN,NaN,NaN,-3.793*** (1.462)
mean_preci,NaN,NaN,NaN,-0.158* (0.087)
dist_100km,NaN,NaN,NaN,-0.011 (2.422)


In [18]:
second_low[[1,2,3,24]]

,1,2,3,24
Intercept,11.305*** (2.557),109.633*** (12.620),112.711*** (12.573),142.286*** (33.644)
predicted,1.258*** (0.152),0.577*** (0.112),0.560*** (0.110),0.601*** (0.152)
Adj. R-squared,0.204,0.208,0.213,0.375
std err,0.152373,0.111869,0.109778,0.152333
longitud,-0.331*** (0.076),-0.267*** (0.079),-0.207** (0.082),-0.279 (0.362)
latitud,NaN,-1.734*** (0.248),-1.870*** (0.250),-2.729*** (0.622)
soil_suitability,NaN,NaN,1.463*** (0.349),1.322*** (0.388)
mean_temp,NaN,NaN,NaN,1.245 (0.987)
mean_preci,NaN,NaN,NaN,0.086 (0.058)
dist_100km,NaN,NaN,NaN,-0.301 (1.635)


In [19]:
second_medium[[1,2,3,24]]

,1,2,3,24
Intercept,28.213*** (1.950),-2.749 (9.504),-3.023 (9.473),8.404 (25.000)
predicted,0.034 (0.116),0.249*** (0.084),0.250*** (0.083),0.155 (0.113)
Adj. R-squared,0.037,0.064,0.069,0.282
std err,0.116192,0.084243,0.082713,0.113194
longitud,-0.245*** (0.058),-0.265*** (0.060),-0.271*** (0.062),0.511* (0.269)
latitud,NaN,0.546*** (0.187),0.558*** (0.189),-0.494 (0.462)
soil_suitability,NaN,NaN,-0.130 (0.263),-0.214 (0.288)
mean_temp,NaN,NaN,NaN,2.452*** (0.733)
mean_preci,NaN,NaN,NaN,0.073* (0.043)
dist_100km,NaN,NaN,NaN,0.389 (1.215)


In [20]:
second_high[[1,2,3,24]]

,1,2,3,24
Intercept,0.864*** (0.186),0.304 (0.916),0.465 (0.913),5.887** (2.361)
predicted,0.015 (0.011),0.019** (0.008),0.018** (0.008),0.031*** (0.011)
Adj. R-squared,0.020,0.025,0.032,0.282
std err,0.011069,0.008123,0.007969,0.010689
longitud,-0.014** (0.006),-0.014** (0.006),-0.011* (0.006),0.003 (0.025)
latitud,NaN,0.010 (0.018),0.003 (0.018),-0.118*** (0.044)
soil_suitability,NaN,NaN,0.077*** (0.025),0.105*** (0.027)
mean_temp,NaN,NaN,NaN,0.096 (0.069)
mean_preci,NaN,NaN,NaN,-0.002 (0.004)
dist_100km,NaN,NaN,NaN,-0.077 (0.115)
